In [ ]:
!pip install dlib flask flask-socketio opencv-python-headless
!pip install keras pyngrok flask_cors
!ngrok config add-authtoken 2pf1VvXV5GbxCciKAzCoXU7mA2i_3WGoRPAUL7sDgcYbGZV1M

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


모델 설정

In [ ]:
import cv2
import dlib
import numpy as np
import base64
from flask import Flask, request, jsonify
from flask_socketio import SocketIO
from flask_cors import CORS
from tensorflow.python.keras.models import load_model
from imutils import face_utils
from pyngrok import ngrok
import os

IMG_SIZE = (34,26)

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("/content/drive/MyDrive/shape_predictor_68_face_landmarks.dat")

model = load_model("/content/drive/MyDrive/2018_12_17_22_58_35.h5")
model.summary()

def crop_eye(img, eye_points):
  x1, y1 = np.amin(eye_points, axis=0)
  x2, y2 = np.amax(eye_points, axis=0)
  cx, cy = (x1 + x2) / 2, (y1 + y2) / 2

  w = (x2 - x1) * 1.2
  h = w * IMG_SIZE[1] / IMG_SIZE[0]

  margin_x, margin_y = w / 2, h / 2

  min_x, min_y = int(cx - margin_x), int(cy - margin_y)
  max_x, max_y = int(cx + margin_x), int(cy + margin_y)

  eye_rect = np.rint([min_x, min_y, max_x, max_y]).astype(np.int)
  eye_img = img[eye_rect[1]:eye_rect[3], eye_rect[0]:eye_rect[2]]

  return eye_img, eye_rect



Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 26, 34, 1)]       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 26, 34, 32)        320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 17, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 13, 17, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 6, 8, 64)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 6, 8, 128)         73856     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 3, 4, 128)         0   

In [ ]:
#얼굴 회전 코드
def align_face(img, shapes):
    """
    얼굴 정렬(기울기 보정)을 수행합니다.
    """
    left_eye_center = np.mean(shapes[36:42], axis=0)
    right_eye_center = np.mean(shapes[42:48], axis=0)

    # 두 눈 사이의 기울기를 계산
    dY = right_eye_center[1] - left_eye_center[1]
    dX = right_eye_center[0] - left_eye_center[0]
    angle = np.degrees(np.arctan2(dY, dX))

    # 중심점과 각도를 기반으로 이미지를 회전
    eyes_center = ((left_eye_center[0] + right_eye_center[0]) // 2,
                   (left_eye_center[1] + right_eye_center[1]) // 2)
    M = cv2.getRotationMatrix2D(eyes_center, angle, 1)
    aligned_img = cv2.warpAffine(img, M, (img.shape[1], img.shape[0]))
    return aligned_img

Flask API 설정

In [ ]:
# Flask 및 SocketIO 설정
app = Flask(__name__)
socketio = SocketIO(app, cors_allowed_origins="*")
CORS(app)
app.config['MAX_CONTENT_LENGTH'] = 16 * 1024 * 1024

# Socket.IO 이벤트 처리
@socketio.on('process_image')
def process_image_socketio(data):
    try:
        img = np.frombuffer(data, np.uint8)
        img = cv2.imdecode(img, cv2.IMREAD_COLOR)

        result = process_image_logic(img)
        socketio.emit('result', result)
    except Exception as e:
        socketio.emit('result', {'status': 'error', 'message': str(e)})

# REST API 엔드포인트 추가
@app.route('/test_process_image', methods=['POST'])
def process_image_rest():
    try:
      if 'image' not in request.files:
        raise ValueError("이미지 파일이 요청에 포함되어 있지 않습니다.")

      file = request.files['image']
      img = np.frombuffer(file.read(), np.uint8)
      img = cv2.imdecode(img, cv2.IMREAD_COLOR)

      result = process_image_logic(img)
      return jsonify(result)
    except Exception as e:
        return jsonify({'status': 'error', 'message': str(e)}), 500

# 공통 이미지 처리 로직
def process_image_logic(data):
    if img is None:
      raise ValueError("이미지 디코딩 실패")

    # 이미지 처리
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = detector(gray)

    eye_closed = 0  # 기본값: 두 눈 뜸
    if len(faces) == 0:
      raise ValueError("얼굴을 감지하지 못함")


    face = faces[0]
    shapes = predictor(gray, face)
    shapes = face_utils.shape_to_np(shapes)


    # 얼굴 정렬
    aligned_img = align_face(img, shapes)
    aligned_gray = cv2.cvtColor(aligned_img, cv2.COLOR_BGR2GRAY)

    # 정렬된 얼굴에서 랜드마크 다시 감지
    aligned_shapes = predictor(aligned_gray, face)
    aligned_shapes = face_utils.shape_to_np(aligned_shapes)

    # 왼쪽과 오른쪽 눈 추출
    eye_img_l, eye_rect_l = crop_eye(aligned_gray, eye_points=aligned_shapes[36:42])
    eye_img_r, eye_rect_r = crop_eye(aligned_gray, eye_points=aligned_shapes[42:48])

    eye_img_l = cv2.resize(eye_img_l, dsize=IMG_SIZE)
    eye_img_r = cv2.resize(eye_img_r, dsize=IMG_SIZE)
    eye_img_r = cv2.flip(eye_img_r, flipCode=1)

    # cv2.imshow('Left Eye', eye_img_l)
    # cv2.imshow('Right Eye', eye_img_r)

    # 눈 상태 예측
    eye_input_l = eye_img_l.copy().reshape((1, IMG_SIZE[1], IMG_SIZE[0], 1)).astype(np.float32) / 255.
    eye_input_r = eye_img_r.copy().reshape((1, IMG_SIZE[1], IMG_SIZE[0], 1)).astype(np.float32) / 255.

    pred_l = model.predict(eye_input_l)
    pred_r = model.predict(eye_input_r)

    # 상태 표시
    state_l = 'O %.1f' if pred_l > 0.1 else '- %.1f'
    state_r = 'O %.1f' if pred_r > 0.1 else '- %.1f'

    state_l = state_l % pred_l
    state_r = state_r % pred_r

    # 결과 시각화
    # cv2.rectangle(aligned_img, pt1=tuple(eye_rect_l[0:2]), pt2=tuple(eye_rect_l[2:4]), color=(255, 255, 255), thickness=2)
    # cv2.rectangle(aligned_img, pt1=tuple(eye_rect_r[0:2]), pt2=tuple(eye_rect_r[2:4]), color=(255, 255, 255), thickness=2)

    # cv2.putText(aligned_img, state_l, tuple(eye_rect_l[0:2]), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
    # cv2.putText(aligned_img, state_r, tuple(eye_rect_r[0:2]), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

    threshold = 0.1
    if pred_l <= threshold and pred_r <= threshold:
        eye_closed = 1  # 두 눈 감음

    return {'status': 'success', 'result': eye_closed}

if __name__ == '__main__':
  public_url = ngrok.connect(5000)
  print(f"ngrok Public URL: {public_url}")

  app.config['DEBUG'] = True

  socketio.run(app, host='0.0.0.0', port=5000, allow_unsafe_werkzeug=True)

ngrok Public URL: NgrokTunnel: "https://3d77-34-45-15-204.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with stat
